# Example notebook to use the ee-water-extraction tool

First, you need to create a Google Cloud Project with the EarthEngine API enabled. 
Then create a service account with the EarthEngine Viewer role, and download a login json key for this service account. 

## Login to Google Cloud

The next cell load Earth Engine package and login to the API using the service account.

In [1]:
import ee

credentials = ee.ServiceAccountCredentials('ee-water-extraction@earthengine-371715.iam.gserviceaccount.com', 
                                           './earthengine-key.json')
ee.Initialize(credentials)

## Classical usage workflow
### Import the required packages

In [56]:
import geemap

from eewaterextraction import data_management
from eewaterextraction import classification
from eewaterextraction import visualization
from eewaterextraction import dgo_metrics

# The next lines are useful if you make modifications in the module files and want to reload it without restarting the notebook kernel
import importlib
importlib.reload(data_management)
importlib.reload(classification)
importlib.reload(visualization)
importlib.reload(dgo_metrics)

<module 'eewaterextraction.dgo_metrics' from 'C:\\Users\\sdunesme\\git\\ee-water-extraction\\eewaterextraction\\dgo_metrics.py'>

### Upload the DGOs to GEE

In [30]:
# Set the local DGO file path
dgo_shapefile_path = './example_data/Lhasa_RC_DGO2km_updated.shp'

# Upload it to GEE
dgo_shp = geemap.shp_to_ee(dgo_shapefile_path)

# Create a region of interset (union of all the DGOs)
roi = dgo_shp.union(1)

### Classify images

In [52]:
# Get the landsat image collection for your ROI
collection = data_management.getLandsatCollection(start=ee.Date('1980-01-01'), 
                                                  end=ee.Date('2100-01-01'), 
                                                  cloud_filter=80, # Maximum cloud coverage accepted (%)
                                                  cloud_masking=True, # Set to False if you don't want to mask the clouds on accepted images
                                                  roi=roi)

# Calculate MNDWI, NDVI and NDWI
collection = classification.calculateIndicators(collection)

# Classify the objects using the indicators
collection = classification.classifyObjects(collection)

### Layers visualisation

At this point of the workflow, you can create an interactive map for an individual landsat image to check all the previously calculated layers.
This feature is only available in a jupyter notebook.

In [57]:
from ipywidgets import interact, fixed

# Get the landsat_product_id of the collection
landsat_ids = collection.aggregate_array('LANDSAT_PRODUCT_ID').getInfo()

map = interact(visualization.imageVisualization, 
         collection=fixed(collection), 
         dgo_shp=fixed(dgo_shp), 
         landsat_id=landsat_ids)

# Show the map in the notebook output
map

interactive(children=(Dropdown(description='landsat_id', options=('LC08_L2SP_137039_20130328_20200912_02_T1', …

<function eewaterextraction.visualization.imageVisualization(collection, landsat_id, dgo_shp)>

### Download layers

If your ROI is not too big, it's possible to download the layers to your local disk. See below the content of each output band.

| band number | content |   
|---|---|
| 1 | blue |
| 2 | green |
| 3 | red |
| 4 | nir |
| 5 | swir1 |
| 6 | swir2 |
| 7 | qa_pixel |
| 8 | mndwi |
| 9 | ndvi |
| 10 | ndwi |
| 11 | water |
| 12 | vegetation |
| 13 | active channel |


In [ ]:
data_management.imageDownload(collection=collection, 
                              landsat_id=landsat_id, 
                              roi=roi, 
                              scale=90, # Downgrading is recommended to reduce the file size
                              output='./example_data/landsat_export.tif')

### Calculate metrics
Execute one of the cell below to select DGOs

In [58]:
# Select some DGOs
dgo_list = [36]
selected_dgo = dgo_shp.filter(ee.Filter.inList('DGO_FID', dgo_list))

In [33]:
# Or select all the DGOs
selected_dgo = dgo_shp

Then calculate the metrics for those DGOs

In [59]:
# Filter dates before metrics calculation
subcollection = collection.filterDate(ee.Date('2011-01-01'), ee.Date('2012-01-01'))

metrics = dgo_metrics.calculateDGOsMetrics(collection=subcollection,
                                           dgos=selected_dgo)

Now you can download calculated metrics as pandas dataframe. Technically, the metric calculation is made at this point of the workflow, so the computation time can be long.

In [ ]:
import pandas as pd

# Get metrics as pandas dataframe
local_data = metrics.getInfo()

props = [improps['properties'] for improps in local_data['features']]
metrics_df = pd.DataFrame(props)

# Save a csv file
metrics_df.to_csv('./example_data/properties.csv')